In [58]:
import pandas as pd
import psycopg2

In [54]:
df = pd.read_csv("/home/tyo/Downloads/Sample-Superstore-Orders.csv")
df

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,6/12/2016,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9989,9990,CA-2014-110422,1/21/2014,1/23/2014,Second Class,TB-21400,Tom Boeckenhauer,Consumer,United States,Miami,...,33180,South,FUR-FU-10001889,Furniture,Furnishings,Ultra Door Pull Handle,25.2480,3,0.20,4.1028
9990,9991,CA-2017-121258,2/26/2017,3/3/2017,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,...,92627,West,FUR-FU-10000747,Furniture,Furnishings,Tenex B1-RE Series Chair Mats for Low Pile Car...,91.9600,2,0.00,15.6332
9991,9992,CA-2017-121258,2/26/2017,3/3/2017,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,...,92627,West,TEC-PH-10003645,Technology,Phones,Aastra 57i VoIP phone,258.5760,2,0.20,19.3932
9992,9993,CA-2017-121258,2/26/2017,3/3/2017,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,...,92627,West,OFF-PA-10004041,Office Supplies,Paper,"It's Hot Message Books with Stickers, 2 3/4"" x 5""",29.6000,4,0.00,13.3200


#### Filtering Data 2

In [52]:
df = df.loc[(df['Ship Mode'] == 'Standard Class') & ((df['City'] == 'Detroit') | (df['City'] == 'Atalanta')), ['Order Date', 'Ship Mode', 'Segment', 'City']]
df

,Order Date,Ship Mode,Segment,City
225,8/9/2015,Standard Class,Corporate,Detroit
226,8/9/2015,Standard Class,Corporate,Detroit
227,8/9/2015,Standard Class,Corporate,Detroit
721,12/15/2016,Standard Class,Corporate,Detroit
722,12/15/2016,Standard Class,Corporate,Detroit
...,...,...,...,...
9149,6/1/2014,Standard Class,Corporate,Detroit
9150,6/1/2014,Standard Class,Corporate,Detroit
9262,10/9/2015,Standard Class,Consumer,Detroit
9263,10/9/2015,Standard Class,Consumer,Detroit


In [61]:
conn = psycopg2.connect(
        host = 'digitalskoladb.c04me33o8tni.ap-southeast-1.rds.amazonaws.com',
        database = 'sandbox',
        user = 'ds_32_g1',
        password = 'natadecoco'
)

In [62]:
curr = conn.cursor()

In [63]:
curr

<cursor object at 0x7f5fb9782020; closed: 0>

In [65]:
sql = """with first_table as(
select 	o1.order_id, shipping_limit_date, price, freight_value,
		o2.customer_id, order_status, order_purchase_timestamp,
		p.product_category_name
from batch_32.order_items_data as o1
left join batch_32.orders_data as o2
on o1.order_id = o2.order_id
left join batch_32.product_data as p
on o1.product_id = p.product_id),
second_table as(
select f.*, c.customer_city, c.customer_state
from first_table as f
left join batch_32.customer_data as c
on f.customer_id = c.customer_id)

select product_category_name, count(distinct order_id) as number_of_order
from second_table
where product_category_name like '%fashion%'
group by product_category_name"""

data = pd.read_sql_query(sql, conn)
data

,product_category_name,number_of_order
0,fashion_bolsas_e_acessorios,1864
1,fashion_calcados,240
2,fashion_esporte,27
3,fashion_roupa_feminina,39
4,fashion_roupa_infanto_juvenil,8
5,fashion_roupa_masculina,112
6,fashion_underwear_e_moda_praia,121
